In [26]:
import numpy as np
from scipy.linalg import svd, norm
from numpy.random import randn, rand
np.random.seed(0)

In [27]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
X[:,3] = X[:,2]
X

array([[ 1.76405235,  0.40015721,  0.97873798,  0.97873798],
       [ 1.86755799, -0.97727788,  0.95008842,  0.95008842],
       [-0.10321885,  0.4105985 ,  0.14404357,  0.14404357],
       [ 0.76103773,  0.12167502,  0.44386323,  0.44386323],
       [ 1.49407907, -0.20515826,  0.3130677 ,  0.3130677 ],
       [-2.55298982,  0.6536186 ,  0.8644362 ,  0.8644362 ]])

In [28]:
# find a vector w in the nullspace of X
w = np.zeros(d)
w[2] = -1
w[3] = 1
X@w

array([0., 0., 0., 0., 0., 0.])

In [29]:
U,S,Vt = svd(X, full_matrices=False)

In [5]:
np.allclose(U@np.diag(S)@Vt, X)

True

In [6]:
U.T@U

array([[ 1.00000000e+00, -7.38697485e-17,  3.23622199e-17,
        -3.99817880e-17],
       [-7.38697485e-17,  1.00000000e+00, -3.81412273e-17,
         5.93941004e-17],
       [ 3.23622199e-17, -3.81412273e-17,  1.00000000e+00,
        -4.16017539e-17],
       [-3.99817880e-17,  5.93941004e-17, -4.16017539e-17,
         1.00000000e+00]])

In [7]:
np.allclose(U.T@U, np.identity(d))

True

In [8]:
U.shape

(6, 4)

In [9]:
U@U.T

array([[ 0.80175178, -0.06399596, -0.00612596,  0.30096884,  0.24817008,
         0.05140258],
       [-0.06399596,  0.9701751 ,  0.00504366,  0.0348882 ,  0.15022773,
         0.03207643],
       [-0.00612596,  0.00504366,  0.9944328 ,  0.05699362, -0.04603767,
        -0.01027116],
       [ 0.30096884,  0.0348882 ,  0.05699362,  0.12011976,  0.0995328 ,
         0.02713897],
       [ 0.24817008,  0.15022773, -0.04603767,  0.0995328 ,  0.15300143,
        -0.18278125],
       [ 0.05140258,  0.03207643, -0.01027116,  0.02713897, -0.18278125,
         0.96051913]])

In [10]:
Vt.shape

(4, 4)

In [11]:
np.allclose(Vt @ Vt.T, np.identity(d))

True

In [12]:
np.allclose(Vt.T @ Vt, np.identity(d))

True

In [13]:
S

array([4.15760175e+00, 2.28949949e+00, 1.01350732e+00, 1.48389401e-16])

In [14]:
# if we have a linearly dependent column, 
# decomposition is just as good if we ignore the 0 in sigma and reduce r by 1
for k in range(d+1):
    print(f"Error of rank {k} approximation: ", 
          np.linalg.norm(X - U[:,:k]@np.diag(S[:k])@(Vt[:k,:])))
    

Error of rank 0 approximation:  4.853314053310529
Error of rank 1 approximation:  2.5037981161489284
Error of rank 2 approximation:  1.0135073191135213
Error of rank 3 approximation:  2.0282945925593685e-15
Error of rank 4 approximation:  2.0404123996834285e-15


In [33]:
# what is a rank 0 approximation?
k = 0
U[:,:k]@np.diag(S[:k])@(Vt[:k,:])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [38]:
# form rank 2 apx of X by zeroing last two singular values
S2 = S.copy()
S2[2:] = 0
U@np.diag(S2)@Vt

array([[ 1.63569576, -0.14072899,  1.04290218,  1.04290218],
       [ 2.03375744, -0.27692435,  0.86700695,  0.86700695],
       [-0.18022995,  0.08607883,  0.18254066,  0.18254066],
       [ 0.71793643, -0.05995099,  0.46540914,  0.46540914],
       [ 1.47764056, -0.27442906,  0.32128515,  0.32128515],
       [-2.51856443,  0.7986849 ,  0.84722729,  0.84722729]])

In [39]:
# form data from noisy linear model
wtrue = randn(d)
y = X@wtrue + .1*randn(n);

In [40]:
# solve least squares problem to estimate w
w4 = Vt.T@np.diag(S**(-1))@U.T@y
w4

array([ 2.32981892e+00, -1.49417792e+00, -1.55210116e+14,  1.55210116e+14])

In [41]:
# it gives a low norm solution, but definitely not optimal...
print("residual given w4:", norm(y - X.dot(w4)))
print("residual given wtrue:", norm(y - X.dot(wtrue)))

residual given w4: 0.3224116203732873
residual given wtrue: 0.3063847433022363


In [42]:
# error in normal equations not zero! uh oh!
norm(X.T@X@w4 - X.T@y)

0.7526118592612931

In [48]:
# use rank k approximation to design matrix X
# k=4 is full rank
# when design matrix X has rank 3, k=3 gives 0 error approximation
# while k=2 results in loss of accuracy
k = 3
w3 = Vt[:k,:].T@np.diag(S[:k]**(-1))@(U[:,:k]).T@y
w3

array([ 2.32981892, -1.45396574, -0.07338535, -0.07338535])

In [49]:
print("residual given w3:", norm(y - X.dot(w3)))
print("error in normal equations given w3:", norm(X.T@X@w3 - X.T@y))

residual given w3: 0.1943391668277365
error in normal equations given w3: 2.175583928816829e-15


In [57]:
# add a vector in the nullspace to w3:
w = w3.copy()
w[2] += 1
w[3] -= 1
norm(X.T@X@w - X.T@y)

1.2560739669470201e-15

Poll:
* A) least squares residual norm(y-Xw) will be higher for w than w3
* B) least squares residual norm(y-Xw) will be lower for w than w3
* C) least squares residual norm(y-Xw) will be the same for w than w3
    
Poll:
* A) there is one global minimum of least squares
* B) there are two global minima of least squares
* C) there are many global minima of least squares
* D) there are infinitely many global minima of least squares

In [55]:
print("residual given w:", norm(y - X.dot(w)))
print("error in normal equations given w:", norm(X.T@X@w - X.T@y))

residual given w: 0.19433916682773522
error in normal equations given w: 7.215595591812694e-15


In [59]:
w

array([ 2.32981892, -1.45396574,  0.92661465, -1.07338535])

In [60]:
# how good is our estimate of w?
norm(w - wtrue) / norm(wtrue)

0.4628663965326436

In [24]:
# compute mean square error
np.mean((y - X@w)**2)

0.006294618627216451

In [61]:
# we can use the numpy.lstsq call instead
w_lstsq = np.linalg.lstsq(X, y, rcond=None)[0]
norm(w_lstsq - w3)

1.0990647210786425e-15